In [ ]:
# imports
import math
import numpy as np
import pandas as pd
from fit import FitEncoder_Weight
from datetime import datetime

### Get the data from Wyze

In the Wyze app, go the the scale and click on the gear in the upper right. Choose Expoer Data and have the data emailed to you.

It will arrive as an XLSX file. Open that in Excel or Numbers and export it as CSV; call it `scale-data.csv`.

### Process the data

Put the CSV in the same directory as this notebook and run the following cells.

NOTE: If you encounter errors, there may be unexpected data in th CSV. Make sure that:

* There are no rows that have non-data entries like `- -`
* Make sure that the CSV file has the column names as the first row, then the data. And nothing else. No “sheet headers” or other kinds of Wyze data at the end.

In [ ]:
# load weight data
rm_lb = lambda x: (float(x.replace('lb','')))
rm_pct = lambda x: (float(x.replace('%','')))
weight_data = pd.read_csv(
    "scale-data.csv",
    parse_dates=['Date and Time'],
    converters={
        "Weight": rm_lb,
        "Body Fat": rm_pct,
        "Muscle Mass": rm_lb,
        "Body Water": rm_pct,
        "Lean Body Mass": rm_lb,
        "Bone Mass": rm_lb,
        "Muscle Mass": rm_lb,
        "Protein": rm_pct,
    },
    dtype={
        "BMI": np.float64,
        "Visceral Fat": np.float64,
        "BMR": np.float64,
    }
)
weight_data.rename(
    inplace = True,
	columns = { 'Date and Time': 'time' }
)

# weight_data


### Select the data you want (optional)

Set the start and end dates for the data you want.

In [ ]:
weight_data =	weight_data[
					(weight_data.time > datetime(2023, 7, 21))
					& 
					(weight_data.time < datetime(2023, 9, 13))
				]
# weight_data

### Create the FIT file

This is one of the formats that Garmin Connect will let you upload.

In [ ]:
fit = FitEncoder_Weight()
fit.write_file_info(time_created = int(weight_data.time.max().timestamp()))
fit.write_file_creator()
fit.write_device_info(timestamp = int(weight_data.time.max().timestamp()))

for i, d in weight_data.iterrows():
    # fit.write_device_info(
    #     timestamp = int(d["time"].timestamp())
    # )
    fit.write_weight_scale(
        timestamp = math.trunc(d["time"].timestamp()),
        weight = float(d["Weight"] * 0.45359237), # convert to kilograms
        percent_fat = d["Body Fat"],
        percent_hydration = d["Body Water"],
        visceral_fat_mass = d["Visceral Fat"],
        bone_mass = d["Bone Mass"],
        muscle_mass = d["Muscle Mass"],
        basal_met = d["BMR"],
        physique_rating = 6,
        active_met = int(d["BMR"] * 1.25),
        metabolic_age = d["Metabolic Age"],
        visceral_fat_rating = d["Body Fat"],
        bmi = d["BMI"]
    )

fit.finish()

# print(fit.getvalue()) # not super useful, but proves there's something in there

try:
    with open("wyze_scale.fit", "wb") as fitfile:
        fitfile.write(fit.getvalue())
        print("Fit file wyze_scale.fit created")
except OSError as e:
    print(f"Got an error: {e}")


### Upload the file

Log in to Garmin Connect and click on the cloud/arrow icon in the upper right and upload this file.